In [21]:
import numpy as np #lineer cebir kütüphanesi
import pandas as pd #veri okuma/ analiz kütüphanesi
import os  #işletim sistemi uyumu için
import re #regular ifade işlemleri için kütüphane
pd.options.mode.chained_assignment = None #uyarıyı görmezden gelmek için
import sys

import nltk #doğal dil işleme kütüphanesi
nltk.download('stopwords')  #stopwords yükleme
from nltk.corpus import stopwords #stopwords işlemleri
from TurkishStemmer import TurkishStemmer #Türkçe stemming işlemleri
import string

from sklearn.model_selection import train_test_split #train test split ayırma
from sklearn.feature_extraction.text import CountVectorizer  #BoW ile vektörleştirme
from sklearn.feature_extraction.text import TfidfVectorizer  #TF-IDF ile vektörleştirme

import matplotlib.pyplot as plt #grafik plot
import seaborn as sns #grafik plot

[nltk_data] Downloading package stopwords to /home/ebru/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
#data tablosunun gösterimi,
data=pd.read_csv('dataall2.csv',encoding="utf8") 
data

,Unnamed: 0,text,label
0,0,Boğaziçi Üniversitesi’nin zengin çocuklarının ...,female
1,1,Eksik Audrey Hepburn’ün hayatını okuyorum. Faş...,female
2,2,Uçağa binmeden yazıyı yetiştireyim demiştim am...,female
3,3,"Somali’de çocukların öldüğü, insanların açlıkt...",female
4,4,Ben neden şimdi öğrendim anne? Laliş (9) okuld...,female
5,5,Ondokuzuncu yüzyılın ortaları. İngiltere’de Yo...,female
6,6,OECD’nin (Ekonomik Kalkınma ve İşbirliği Örgüt...,female
7,7,"İnsanlar bir yana, şu sıralar Kıbrıs’ta kedile...",female
8,8,Gideceğinizi bildiğiniz bir yerde kalmanızda h...,female
9,9,Üniversitedeyken İranlı bir arkadaşım vardı: M...,female


In [23]:
#csv yapıp tekrar okutmadan kaynaklanan gereksiz column u silme
data = data.drop(['Unnamed: 0'],axis=1)

,text,label
478,Aptal yerine konmanın bu kadarı da olmaz Can A...,male
479,İlk Üç Madde 15 Ekim 2011 Cumhuriyet 1982 Anay...,male
480,Bebek ağlaması İnsanı en çaresiz bırakan bir b...,male
481,AKP’nin işine gelmezse Özel Yetkili Savcılık k...,male
482,Bugün 12 Eylül Bugün 12 Eylül 1980 darbesinin ...,male


In [28]:
#index kontrolü
data.tail()

,text,label
478,Aptal yerine konmanın bu kadarı da olmaz Can A...,male
479,İlk Üç Madde 15 Ekim 2011 Cumhuriyet 1982 Anay...,male
480,Bebek ağlaması İnsanı en çaresiz bırakan bir b...,male
481,AKP’nin işine gelmezse Özel Yetkili Savcılık k...,male
482,Bugün 12 Eylül Bugün 12 Eylül 1980 darbesinin ...,male


In [29]:
#NLP işlemleri için text colomnu yeni dataframe e atandı
df = data[['text']]
df

,text
0,Boğaziçi Üniversitesi’nin zengin çocuklarının ...
1,Eksik Audrey Hepburn’ün hayatını okuyorum. Faş...
2,Uçağa binmeden yazıyı yetiştireyim demiştim am...
3,"Somali’de çocukların öldüğü, insanların açlıkt..."
4,Ben neden şimdi öğrendim anne? Laliş (9) okuld...
5,Ondokuzuncu yüzyılın ortaları. İngiltere’de Yo...
6,OECD’nin (Ekonomik Kalkınma ve İşbirliği Örgüt...
7,"İnsanlar bir yana, şu sıralar Kıbrıs’ta kedile..."
8,Gideceğinizi bildiğiniz bir yerde kalmanızda h...
9,Üniversitedeyken İranlı bir arkadaşım vardı: M...


In [30]:
#NLP işlemleri
#gereksiz boşlukların kaldırılması
df['text'] = (df['text'].str.split()).str.join(' ')
df.head()

,text
0,Boğaziçi Üniversitesi’nin zengin çocuklarının ...
1,Eksik Audrey Hepburn’ün hayatını okuyorum. Faş...
2,Uçağa binmeden yazıyı yetiştireyim demiştim am...
3,"Somali’de çocukların öldüğü, insanların açlıkt..."
4,Ben neden şimdi öğrendim anne? Laliş (9) okuld...


In [31]:
#harfleri küçültme
df["text_lower"] = df['text'].str.lower()
df.head()

,text,text_lower
0,Boğaziçi Üniversitesi’nin zengin çocuklarının ...,boğaziçi üniversitesi’nin zengin çocuklarının ...
1,Eksik Audrey Hepburn’ün hayatını okuyorum. Faş...,eksik audrey hepburn’ün hayatını okuyorum. faş...
2,Uçağa binmeden yazıyı yetiştireyim demiştim am...,uçağa binmeden yazıyı yetiştireyim demiştim am...
3,"Somali’de çocukların öldüğü, insanların açlıkt...","somali’de çocukların öldüğü, insanların açlıkt..."
4,Ben neden şimdi öğrendim anne? Laliş (9) okuld...,ben neden şimdi öğrendim anne? laliş (9) okuld...


In [32]:
#işleme etkisi olmayan noktama işaretleri kaldırılır
#string punc'un içindeki default noktalama işaretleri dikkate alır, ayrıca kesme işareti eklendi
kaldirilan_noktalamalar = string.punctuation+u"’"
#noktalama kaldırma fonksiyonu
def noktalamaKaldir(text):
    return text.translate(str.maketrans('','',kaldirilan_noktalamalar))


In [33]:
#noktalama işaretleri kaldırma
df["text_isaretsiz"] = df["text_lower"].apply(lambda text: noktalamaKaldir(text))
df.head()

,text,text_lower,text_isaretsiz
0,Boğaziçi Üniversitesi’nin zengin çocuklarının ...,boğaziçi üniversitesi’nin zengin çocuklarının ...,boğaziçi üniversitesinin zengin çocuklarının g...
1,Eksik Audrey Hepburn’ün hayatını okuyorum. Faş...,eksik audrey hepburn’ün hayatını okuyorum. faş...,eksik audrey hepburnün hayatını okuyorum faşis...
2,Uçağa binmeden yazıyı yetiştireyim demiştim am...,uçağa binmeden yazıyı yetiştireyim demiştim am...,uçağa binmeden yazıyı yetiştireyim demiştim am...
3,"Somali’de çocukların öldüğü, insanların açlıkt...","somali’de çocukların öldüğü, insanların açlıkt...",somalide çocukların öldüğü insanların açlıktan...
4,Ben neden şimdi öğrendim anne? Laliş (9) okuld...,ben neden şimdi öğrendim anne? laliş (9) okuld...,ben neden şimdi öğrendim anne laliş 9 okuldan ...


In [34]:
#işleme etkisi olmayan kelimeler kaldırılır
etkisiz_kelimeler = set(stopwords.words('turkish'))
#stopwords kaldırma fonksiyonu
def etkisizleriKaldir(text):
    return " ".join([word for word in str(text).split() if word not in etkisiz_kelimeler])

In [35]:
#stopwords kaldırma
df["text_stopsuz"] = df["text_isaretsiz"].apply(lambda text : etkisizleriKaldir(text))
df.head()

,text,text_lower,text_isaretsiz,text_stopsuz
0,Boğaziçi Üniversitesi’nin zengin çocuklarının ...,boğaziçi üniversitesi’nin zengin çocuklarının ...,boğaziçi üniversitesinin zengin çocuklarının g...,boğaziçi üniversitesinin zengin çocuklarının g...
1,Eksik Audrey Hepburn’ün hayatını okuyorum. Faş...,eksik audrey hepburn’ün hayatını okuyorum. faş...,eksik audrey hepburnün hayatını okuyorum faşis...,eksik audrey hepburnün hayatını okuyorum faşis...
2,Uçağa binmeden yazıyı yetiştireyim demiştim am...,uçağa binmeden yazıyı yetiştireyim demiştim am...,uçağa binmeden yazıyı yetiştireyim demiştim am...,uçağa binmeden yazıyı yetiştireyim demiştim yi...
3,"Somali’de çocukların öldüğü, insanların açlıkt...","somali’de çocukların öldüğü, insanların açlıkt...",somalide çocukların öldüğü insanların açlıktan...,somalide çocukların öldüğü insanların açlıktan...
4,Ben neden şimdi öğrendim anne? Laliş (9) okuld...,ben neden şimdi öğrendim anne? laliş (9) okuld...,ben neden şimdi öğrendim anne laliş 9 okuldan ...,ben şimdi öğrendim anne laliş 9 okuldan geldi ...


In [38]:
#kelime kelime ayırma işlemi (tokenize)
df['tokenize'] = df['text_stopsuz'].str.split() #yazıdaki tüm kelimeleri ayırma işlemi


In [39]:
#stemmer, ayrılan her kelimenin kökünü bulma işlemi
stemmer = TurkishStemmer()
df['stemmed'] = df['tokenize'].apply(lambda x: [stemmer.stem(y) for y in x]) 
df.head()

,text,text_lower,text_isaretsiz,text_stopsuz,tokenize,stemmed
0,Boğaziçi Üniversitesi’nin zengin çocuklarının ...,boğaziçi üniversitesi’nin zengin çocuklarının ...,boğaziçi üniversitesinin zengin çocuklarının g...,boğaziçi üniversitesinin zengin çocuklarının g...,"[boğaziçi, üniversitesinin, zengin, çocukların...","[boğaziç, üniversite, zeng, çocuk, gittik, bir..."
1,Eksik Audrey Hepburn’ün hayatını okuyorum. Faş...,eksik audrey hepburn’ün hayatını okuyorum. faş...,eksik audrey hepburnün hayatını okuyorum faşis...,eksik audrey hepburnün hayatını okuyorum faşis...,"[eksik, audrey, hepburnün, hayatını, okuyorum,...","[eksik, audrey, hepburnü, hayat, okuyor, faşis..."
2,Uçağa binmeden yazıyı yetiştireyim demiştim am...,uçağa binmeden yazıyı yetiştireyim demiştim am...,uçağa binmeden yazıyı yetiştireyim demiştim am...,uçağa binmeden yazıyı yetiştireyim demiştim yi...,"[uçağa, binmeden, yazıyı, yetiştireyim, demişt...","[uçak, binme, yazı, yetiştir, demiş, yin, yolc..."
3,"Somali’de çocukların öldüğü, insanların açlıkt...","somali’de çocukların öldüğü, insanların açlıkt...",somalide çocukların öldüğü insanların açlıktan...,somalide çocukların öldüğü insanların açlıktan...,"[somalide, çocukların, öldüğü, insanların, açl...","[somali, çocuk, öldük, insan, açlık, kırıldık,..."
4,Ben neden şimdi öğrendim anne? Laliş (9) okuld...,ben neden şimdi öğrendim anne? laliş (9) okuld...,ben neden şimdi öğrendim anne laliş 9 okuldan ...,ben şimdi öğrendim anne laliş 9 okuldan geldi ...,"[ben, şimdi, öğrendim, anne, laliş, 9, okuldan...","[ben, şimt, öğren, anne, laliş, 9, okul, gelt,..."


In [40]:
#data framein son hali
data["text"] = df['stemmed']
data.head()

,text,label
0,"[boğaziç, üniversite, zeng, çocuk, gittik, bir...",female
1,"[eksik, audrey, hepburnü, hayat, okuyor, faşis...",female
2,"[uçak, binme, yazı, yetiştir, demiş, yin, yolc...",female
3,"[somali, çocuk, öldük, insan, açlık, kırıldık,...",female
4,"[ben, şimt, öğren, anne, laliş, 9, okul, gelt,...",female


In [41]:
data.to_csv("stemmed.csv")